In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.backend as K
import tensorboard
import shutil

from datetime import datetime as dt

In [ ]:
BASE_PATH = "gdrive/My Drive/kaggle/"

train_csv = pd.read_csv(BASE_PATH + "train.csv")
test_csv = pd.read_csv(BASE_PATH + "test.csv")

train_csv['Patient_Week'] = train_csv['Patient'].astype(str) + '_' + train_csv['Weeks'].astype(str)

train_data = train_csv.groupby("Patient")
# print(len(train_data))

unique_patients = train_csv['Patient'].unique()
# print(unique_patients)

for patient_id in unique_patients:
    print(train_csv[train_csv['Patient']==patient_id].iloc[0])
    break

In [ ]:
# Need to add base fvc column, base percentage
train_csv['Weeks'] = train_csv['Weeks'].astype(int)
train_csv['base_week'] = train_csv.groupby('Patient')['Weeks'].transform('min')
train_csv['weeks_passed'] = train_csv['Weeks'] - train_csv['base_week']
train_csv['base_fvc'] = 0
train_csv['base_percent'] = 0

for index in range(len(train_csv)):
    base_data = train_csv[train_csv['Patient']==train_csv.iloc[index]['Patient']].iloc[0]
    train_csv.loc[index, 'base_fvc'] = base_data['FVC']
    train_csv.loc[index, 'base_percent'] = base_data['Percent']
    
train_csv

In [ ]:
# X_data = train_csv[['Sex', 'SmokingStatus']]
X_data = train_csv[['Sex', 'SmokingStatus']]
X_data = pd.get_dummies(data=X_data, drop_first=True)
selected_columns = ['Age', 'weeks_passed', 'base_fvc', 'base_percent']
X_data[selected_columns] = train_csv[selected_columns]
y_data = train_csv['FVC']

X_train = X_data.sample(frac=0.8,random_state=0)
X_test = X_data.drop(X_train.index)

y_train = y_data.sample(frac=0.8,random_state=0).astype(float)
y_test = y_data.drop(y_train.index).astype(float)

train_stats = X_train.describe().transpose()

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']


normalized_X_train = norm(X_train)
normalized_X_test = norm(X_test)

In [ ]:
# sns.pairplot(X_train[['Age', 'weeks_passed', 'base_fvc', 'base_percent']], diag_kind="kde")

In [ ]:
def laplace_log_likelihood(actual_fvc, predicted_fvc, confidence, return_values = False):
    """
    Calculates the modified Laplace Log Likelihood score for this competition.
    """
    sd_clipped = np.maximum(confidence, 70)
    delta = np.minimum(np.abs(actual_fvc - predicted_fvc), 1000)
    metric = - np.sqrt(2) * delta / sd_clipped - np.log(np.sqrt(2) * sd_clipped)

    if return_values:
        return metric
    else:
        return np.mean(metric)

In [ ]:
# def log_custom_acc(y_test, y_pred):
#     sd_clipped = tf.maximum(tf.math.reduce_std(y_pred), 70)
#     delta = tf.minimum(tf.math.abs(tf.math.subtract(y_test, y_pred)), 1000)
    
#     metric = - tf.math.sqrt(2) * delta / sd_clipped - tf.math.log(np.sqrt(2) * sd_clipped)
#     return metric

def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)

In [ ]:
# Loss - laplace likelihood

def log_custom_loss(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    
    fvc_pred = y_pred
    sigma = tf.math.reduce_std(y_pred)
    
    sigma_clip = tf.maximum(sigma, 70)
    delta = tf.abs(y_true - fvc_pred)
    delta = tf.minimum(delta, 1000)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32))
    metric = (delta / sigma_clip) * sq2 + tf.math.log(sigma_clip * sq2)
    return K.mean(metric)

  
def laplace_log_metric(y_true, y_pred):
  return -log_custom_loss(y_true, y_pred)

In [ ]:
# For quantile
def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[len(X_train.keys())]),
    layers.Dropout(0.25),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)
#   quantile=0.5
  
#   loss=tf.keras.losses.MeanSquaredLogarithmicError()
#   loss=lambda y,f: tilted_loss(quantile,y,f)

  model.compile(loss=log_custom_loss,
                optimizer=optimizer,
                metrics=[laplace_log_metric, 'mae', 'mse'])
  return model

model = build_model()

In [ ]:
model.summary()

In [ ]:
# shutil.rmtree('logs')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "gdrive/My Drive/kaggle/logs"

In [ ]:
EPOCHS = 1000

time_now = dt.now().strftime("%Y%m%d_%H%M%S")
my_callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=BASE_PATH + 'logs/'+str(time_now)),
]

history = model.fit(
  normalized_X_train, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=1, callbacks=my_callbacks)

In [ ]:
def inference(model, X_test,y_test):
    y_pred = model.predict(X_test).reshape(-1)
    return y_pred, laplace_log_likelihood(y_test, y_pred, np.std(y_pred))

y_pred, metric = inference(model, normalized_X_test, y_test.tolist())
print(metric)

In [ ]:
np.std(y_pred)

In [ ]:
loss, mae, mse = model.evaluate(normalized_X_test, y_test, verbose=2)